In [2]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración

# Pair programming Nulos

## Ejercicios gestión de nulos

En este pair programming usaremos el csv que generamos en el pair programming de ayer. El objetivo del ejercicio es identificar las columnas que contienen valores nulos y aplicar un proceso de imputación para manejarlos de manera adecuada.

Instrucciones:

1. Identificación de Valores Nulos: Identifica todas las columnas que contengan valores nulos en el DataFrame.


In [14]:
# cargamos el dataframe creado en la lección anterior
df = pd.read_csv("./df_world2.csv" ,index_col = 0)
df.head()

,Unnamed: 0.1,country,density,abbreviation,agriculturalland,landarea,armedforcessize,birthrate,callingcode,capital/majorcity,...,outofpockethealthexpenditure,physiciansperthousand,population,population:laborforceparticipation,taxrevenue,totaltaxrate,unemploymentrate,urban_population,latitud,longitud
0,0,Afghanistan,60,AF,58.1,"652,230","323,000",32.49,93.0,Kabul,...,78.4,0.28,"38,041,754",48.9,9.3,71.4,11.12,"9,797,273",33.93911,67.709953
1,1,Albania,105,AL,43.1,"28,748","9,000",11.78,355.0,Tirana,...,56.9,1.20,"2,854,191",55.7,18.6,36.6,12.33,"1,747,593",41.153332,20.168331
2,2,Algeria,18,DZ,17.4,"2,381,741","317,000",24.28,213.0,Algiers,...,28.1,1.72,"43,053,054",41.2,37.2,66.1,11.70,"31,510,100",28.033886,1.659626
3,3,Andorra,164,AD,40.0,468,NaN,7.20,376.0,Andorra la Vella,...,36.4,3.33,"77,142",NaN,NaN,NaN,NaN,"67,873",42.506285,1.521801
4,4,Angola,26,AO,47.5,"1,246,700","117,000",40.73,244.0,Luanda,...,33.4,0.21,"31,825,295",77.5,9.2,49.1,6.89,"21,061,025",-11.202692,17.873887


In [15]:
print(df.to_string())

     Unnamed: 0.1                           country density abbreviation  agriculturalland    landarea armedforcessize  birthrate  callingcode       capital/majorcity co2-emissions       cpi  cpichange currency-code  fertilityrate  forestedarea  gasolineprice                  gdp  grossprimaryeducationenrollment  grosstertiaryeducationenrollment  infantmortality              largestcity  lifeexpectancy  maternalmortalityratio  minimumwage        officiallanguage  outofpockethealthexpenditure  physiciansperthousand     population  population:laborforceparticipation  taxrevenue  totaltaxrate  unemploymentrate urban_population      latitud    longitud
0               0                       Afghanistan      60           AF              58.1     652,230         323,000      32.49         93.0                   Kabul         8,672     149.9        2.3           AFN           4.47           2.1           0.70      19,101,353,833                             104.0                              

In [21]:
df.dtypes


Unnamed: 0.1                            int64
country                                object
density                                object
abbreviation                           object
agriculturalland                      float64
landarea                               object
armedforcessize                        object
birthrate                             float64
callingcode                           float64
capital/majorcity                      object
co2-emissions                          object
cpi                                    object
cpichange                             float64
currency-code                          object
fertilityrate                         float64
forestedarea                          float64
gasolineprice                         float64
gdp                                    object
grossprimaryeducationenrollment       float64
grosstertiaryeducationenrollment      float64
infantmortality                       float64
largestcity                       

In [31]:
numericos = []
categoricos = []

for col in df.columns:
    if df.dtypes[col] == object:
        categoricos.append(col)
    else:
        numericos.append(col)

print("Columnas numéricas:", numericos)
print("Columnas categóricas:", categoricos)

Columnas numéricas: ['Unnamed: 0.1', 'agriculturalland', 'birthrate', 'callingcode', 'cpichange', 'fertilityrate', 'forestedarea', 'gasolineprice', 'grossprimaryeducationenrollment', 'grosstertiaryeducationenrollment', 'infantmortality', 'lifeexpectancy', 'maternalmortalityratio', 'minimumwage', 'outofpockethealthexpenditure', 'physiciansperthousand', 'population:laborforceparticipation', 'taxrevenue', 'totaltaxrate', 'unemploymentrate', 'longitud']
Columnas categóricas: ['country', 'density', 'abbreviation', 'landarea', 'armedforcessize', 'capital/majorcity', 'co2-emissions', 'cpi', 'currency-code', 'gdp', 'largestcity', 'officiallanguage', 'population', 'urban_population', 'latitud']


In [32]:
print(numericos)


['Unnamed: 0.1', 'agriculturalland', 'birthrate', 'callingcode', 'cpichange', 'fertilityrate', 'forestedarea', 'gasolineprice', 'grossprimaryeducationenrollment', 'grosstertiaryeducationenrollment', 'infantmortality', 'lifeexpectancy', 'maternalmortalityratio', 'minimumwage', 'outofpockethealthexpenditure', 'physiciansperthousand', 'population:laborforceparticipation', 'taxrevenue', 'totaltaxrate', 'unemploymentrate', 'longitud']


In [33]:
print("Columnas categóricas:", categoricos)

Columnas categóricas: ['country', 'density', 'abbreviation', 'landarea', 'armedforcessize', 'capital/majorcity', 'co2-emissions', 'cpi', 'currency-code', 'gdp', 'largestcity', 'officiallanguage', 'population', 'urban_population', 'latitud']


In [11]:
# lo convertimos a DataFrame
df_nulos_ba = pd.DataFrame((df.isnull().sum() / df.shape[0]) * 100, columns = ["%_nulos"])

# filtramos el DataFrame para quedarnos solo con aquellas columnas que tengan nulos
df_nulos_ba[df_nulos_ba["%_nulos"] > 0]

,%_nulos
abbreviation,3.589744
agriculturalland,3.589744
landarea,0.512821
armedforcessize,12.307692
birthrate,3.076923
callingcode,0.512821
capital/majorcity,1.538462
co2-emissions,3.589744
cpi,8.717949
cpichange,8.205128


In [38]:
df_nume= df[['Unnamed: 0.1', 'agriculturalland', 'birthrate', 'callingcode', 'cpichange', 'fertilityrate', 'forestedarea', 'gasolineprice', 'grossprimaryeducationenrollment', 'grosstertiaryeducationenrollment', 'infantmortality', 'lifeexpectancy', 'maternalmortalityratio', 'minimumwage', 'outofpockethealthexpenditure', 'physiciansperthousand', 'population:laborforceparticipation', 'taxrevenue', 'totaltaxrate', 'unemploymentrate', 'longitud']]

In [39]:
df_nume

,Unnamed: 0.1,agriculturalland,birthrate,callingcode,cpichange,fertilityrate,forestedarea,gasolineprice,grossprimaryeducationenrollment,grosstertiaryeducationenrollment,...,lifeexpectancy,maternalmortalityratio,minimumwage,outofpockethealthexpenditure,physiciansperthousand,population:laborforceparticipation,taxrevenue,totaltaxrate,unemploymentrate,longitud
0,0,58.1,32.49,93.0,2.3,4.47,2.1,0.70,104.0,9.7,...,64.5,638.0,0.43,78.4,0.28,48.9,9.3,71.4,11.12,67.709953
1,1,43.1,11.78,355.0,1.4,1.62,28.1,1.36,107.0,55.0,...,78.5,15.0,1.12,56.9,1.20,55.7,18.6,36.6,12.33,20.168331
2,2,17.4,24.28,213.0,2.0,3.02,0.8,0.28,109.9,51.4,...,76.7,112.0,0.95,28.1,1.72,41.2,37.2,66.1,11.70,1.659626
3,3,40.0,7.20,376.0,NaN,1.27,34.0,1.51,106.4,NaN,...,NaN,NaN,6.63,36.4,3.33,NaN,NaN,NaN,NaN,1.521801
4,4,47.5,40.73,244.0,17.1,5.52,46.3,0.97,113.5,9.3,...,60.8,241.0,0.71,33.4,0.21,77.5,9.2,49.1,6.89,17.873887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,190,24.5,17.88,58.0,254.9,2.27,52.7,0.00,97.2,79.3,...,72.1,125.0,0.01,45.8,1.92,59.7,NaN,73.3,8.80,-66.589730
191,191,39.3,16.75,84.0,2.8,2.05,48.1,0.80,110.6,28.5,...,75.3,43.0,0.73,43.5,0.82,77.4,19.1,37.6,2.01,108.277199
192,192,44.6,30.45,967.0,8.1,3.79,1.0,0.92,93.6,10.2,...,66.1,164.0,NaN,81.0,0.31,38.0,NaN,26.6,12.91,48.516388
193,193,32.1,36.19,260.0,9.2,4.63,65.2,1.40,98.7,4.1,...,63.5,213.0,0.24,27.5,1.19,74.6,16.2,15.6,11.43,27.849332


In [44]:
(df_nume.isnull().sum() / df_nume.shape[0]) * 100

Unnamed: 0.1                           0.000000
agriculturalland                       3.589744
birthrate                              3.076923
callingcode                            0.512821
cpichange                              8.205128
fertilityrate                          3.589744
forestedarea                           3.589744
gasolineprice                         10.256410
grossprimaryeducationenrollment        3.589744
grosstertiaryeducationenrollment       6.153846
infantmortality                        3.076923
lifeexpectancy                         4.102564
maternalmortalityratio                 7.179487
minimumwage                           23.076923
outofpockethealthexpenditure           3.589744
physiciansperthousand                  3.589744
population:laborforceparticipation     9.743590
taxrevenue                            13.333333
totaltaxrate                           6.153846
unemploymentrate                       9.743590
longitud                               0

In [40]:
df_nume.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0.1,195.0,97.000000,56.435804,0.000000,48.500000,97.000000,145.500000,194.000000
agriculturalland,188.0,39.117553,21.783052,0.600000,21.700000,39.600000,55.375000,82.600000
birthrate,189.0,20.214974,9.945774,5.900000,11.300000,17.950000,28.750000,46.080000
callingcode,194.0,360.546392,323.236419,1.000000,82.500000,255.500000,506.750000,1876.000000
cpichange,179.0,6.722346,24.450414,-4.300000,1.000000,2.300000,4.250000,254.900000
fertilityrate,188.0,2.698138,1.282267,0.980000,1.705000,2.245000,3.597500,6.910000
forestedarea,188.0,32.015426,23.791331,0.000000,11.000000,32.000000,48.175000,98.300000
gasolineprice,175.0,1.002457,0.368858,0.000000,0.755000,0.980000,1.240000,2.000000
grossprimaryeducationenrollment,188.0,102.470213,13.151607,23.400000,98.950000,102.550000,108.025000,142.500000
grosstertiaryeducationenrollment,183.0,37.963388,29.261022,0.800000,12.150000,31.200000,63.150000,136.600000


In [12]:
# Obtenemos la lista de columnas categóricas que tienen nulos
nulos_esta_cat = df[df.columns[df.isnull().any()]].select_dtypes(include = "O").columns
print("Las columnas categóricas que tienen nulos son : \n ")
print(nulos_esta_cat)

Las columnas categóricas que tienen nulos son : 
 
Index(['abbreviation', 'landarea', 'armedforcessize', 'capital/majorcity',
       'co2-emissions', 'cpi', 'currency-code', 'gdp', 'largestcity',
       'officiallanguage', 'population', 'urban_population'],
      dtype='object')


In [47]:
categoricos

['country',
 'density',
 'abbreviation',
 'landarea',
 'armedforcessize',
 'capital/majorcity',
 'co2-emissions',
 'cpi',
 'currency-code',
 'gdp',
 'largestcity',
 'officiallanguage',
 'population',
 'urban_population',
 'latitud']

In [13]:
# sacamos el 'value_counts()' de cada una de las columnas categóricas que tienen nulos para saber como es la distribución de sus categorías
for col in nulos_esta_cat:
    print(f"La distribución de las categorías para la columna {col.upper()}")
    display(df[col].value_counts() / df.shape[0])
    print("........................")

La distribución de las categorías para la columna ABBREVIATION


AF    0.005128
PY    0.005128
NE    0.005128
NG    0.005128
KP    0.005128
        ...   
GR    0.005128
GD    0.005128
GT    0.005128
GN    0.005128
ZW    0.005128
Name: abbreviation, Length: 188, dtype: float64

........................
La distribución de las categorías para la columna LANDAREA


652,230      0.005128
616          0.005128
268,838      0.005128
130,370      0.005128
1,267,000    0.005128
               ...   
349          0.005128
108,889      0.005128
245,857      0.005128
36,125       0.005128
390,757      0.005128
Name: landarea, Length: 194, dtype: float64

........................
La distribución de las categorías para la columna ARMEDFORCESSIZE


...

........................
La distribución de las categorías para la columna POPULATION


38,041,754    0.005128
182,790       0.005128
4,841,000     0.005128
6,545,502     0.005128
23,310,715    0.005128
                ...   
112,003       0.005128
16,604,026    0.005128
12,771,246    0.005128
1,920,922     0.005128
14,645,468    0.005128
Name: population, Length: 194, dtype: float64

........................
La distribución de las categorías para la columna URBAN_POPULATION


9,797,273      0.005128
1,162,834      0.005128
3,846,137      0.005128
3,850,231      0.005128
102,806,948    0.005128
                 ...   
8,507,474      0.005128
40,765         0.005128
8,540,945      0.005128
4,661,505      0.005128
4,717,305      0.005128
Name: urban_population, Length: 190, dtype: float64

........................



2. Selección de Método de Imputación: Discute en tu compañera cuál sería la mejor estrategia para manejar los valores nulos en cada una de las columnas identificadas en el paso anterior.


In [ ]:
 Obtenemos la lista de columnas categóricas que tienen nulos
nulos_esta_cat = df[df.columns[df.isnull().any()]].select_dtypes(include = "O").columns
print("Las columnas categóricas que tienen nulos son : \n ")
print(nulos_esta_cat)
# sacamos el 'value_counts()' de cada una de las columnas categóricas que tienen nulos para saber como es la distribución de sus categorías
for col in nulos_esta_cat:
    print(f"La distribución de las categorías para la columna {col.upper()}")
    display(df[col].value_counts() / df.shape[0])

In [46]:
columnas_media = df_nume[["gasolineprice","minimumwage"]]
# iteramos por la lista creada en el paso anterior:
for columna in columnas_media:
    
    # calculamos la moda para la columna por la que estamos iterando
    media = df_nume[columna].mean()[0]    
    
    # utilizando el método fillna reemplazamos los valores nulos por la moda calculada en el paso anterior. 
    df_nume[columna] = df_nume[columna].fillna(media)
# por último chequeamos si se han eliminado los nulos en las columnas de "marital" y "loan"
print("Después del reemplazo usando 'fillna' quedan los siguientes nulos")
df_nume[columnas_media].isnull().sum()

TypeError: 'float' object is not subscriptable

In [ ]:
media = df_nume[col].mean()[0]

df_nume[columna] = df_mume[columna].fillna(moda)


3. Imputación de Valores Nulos: Implementa el método de imputación seleccionado en el paso 2 para llenar los valores nulos en las columnas.



4. Informe: Añade al final de un jupyter una explicación breve que describa las columnas que tenían valores nulos, cómo decidiste imputarlos y cualquier observación adicional que consideres importante sobre el proceso de limpieza de datos.
